In [1]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [3]:
#DataFrame der CSU
df_csu_raw = pd.read_excel("../data/Parteien/@CSU_user_tweets_bearbeitet.xlsx")
df_csu_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
tweets_df_clean = df_csu_raw[['Name', 'original_tweets']]




In [57]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Create a new column with emji free text 
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets
0,CSU,Wir gratulieren unserem Bundesverkehrsminister Andreas Scheuer herzlichst zu seinem heutigen Geburtstag! https://t.co/JIwPQwKZpp,Wir gratulieren unserem Bundesverkehrsminister Andreas Scheuer herzlichst zu seinem heutigen Geburtstag! https://t.co/JIwPQwKZpp,wir gratulieren unserem bundesverkehrsminister andreas scheuer herzlichst zu seinem heutigen geburtstag httpstcojiwpqwkzpp
1,CSU,An alle unsere Wahlkämpfer und Wähler: Danke für eure Unterstützung! #btw21 https://t.co/3lL8q9PJzR,An alle unsere Wahlkämpfer und Wähler: Danke für eure Unterstützung! #btw21 https://t.co/3lL8q9PJzR,an alle unsere wahlkämpfer und wähler danke für eure unterstützung
2,CSU,Damit Bayern in Deutschland stark bleibt! Heute mit beiden Stimmen CSU! #btw21 https://t.co/WKjCFCmjiT,Damit Bayern in Deutschland stark bleibt! Heute mit beiden Stimmen CSU! #btw21 https://t.co/WKjCFCmjiT,damit bayern in deutschland stark bleibt heute mit beiden stimmen csu httpstcowkjcfcmjit
3,CSU,"Mit spitzer Feder gegen den Linksrutsch: Alexander Dobrindt trifft @janfleischhauer. Schaut rein in das Gespräch ÜBER statt UNTER LINKEN - und lernt von Jan Fleischhauer, was durchscholzen bedeutet. Jetzt unterstützen und Linksrutsch verhindern: https://t.co/fZun4YDRny https://t.co/8QXxDWPVr6","Mit spitzer Feder gegen den Linksrutsch: Alexander Dobrindt trifft @janfleischhauer. Schaut rein in das Gespräch ÜBER statt UNTER LINKEN - und lernt von Jan Fleischhauer, was durchscholzen bedeutet. Jetzt unterstützen und Linksrutsch verhindern: https://t.co/fZun4YDRny https://t.co/8QXxDWPVr6",mit spitzer feder gegen den linksrutsch alexander dobrindt trifft janfleischhauer schaut rein in das gespräch über statt unter linken und lernt von jan fleischhauer was durchscholzen bedeutet jetzt unterstützen und linksrutsch verhindern
4,CSU,"Wir wollen, dass Armin Laschet der nächste Kanzler wird! Damit Deutschland stabil bleibt! #btw21 https://t.co/ZIP4PsASsQ","Wir wollen, dass Armin Laschet der nächste Kanzler wird! Damit Deutschland stabil bleibt! #btw21 https://t.co/ZIP4PsASsQ",wir wollen dass armin laschet der nächste kanzler wird damit deutschland stabil bleibt
...,...,...,...,...
317,CSU,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,gedenkminute im stadion für die opfer der entsetzlichen messerattacke in würzburg wir sind in gedanken bei den familien
318,CSU,🔵 Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. ⬇️ https://t.co/yk6Nvr90xW,Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. https://t.co/yk6Nvr90xW,jetzt live die aufstellung der csuliste für die bundestagswahl aus dem maxmorlockstadtion in nürnberg
319,CSU,".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so etwas in Zukunft zu verhindern! #Würzburg",".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so etwas in Zukunft zu verhindern! #Würzburg",markussoeder ich danke der polizei für ihren schnellen einsatz besonders beeindruckt hat mich das engagement der bürger wir stehen nicht nur den betroffenen bei sondern werden alles tun um so etwas in zukunft zu verhindern würzburg
320,CSU,".@Markus_Soeder: Die Ereignisse von #Würzburg sind unfassbar und schockierend. Wir trauern mit den Opfern, wir bangen, beten und hoffen mit den Angehörigen. In diesen schwierigen Zeiten sind wir alle solidarisch! Ich werde heute #Trauerbeflaggung für 

In [58]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [59]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

bundesverkehrsminister heutig geburtstag


In [60]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [61]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,CSU,Wir gratulieren unserem Bundesverkehrsminister Andreas Scheuer herzlichst zu seinem heutigen Geburtstag! https://t.co/JIwPQwKZpp,Wir gratulieren unserem Bundesverkehrsminister Andreas Scheuer herzlichst zu seinem heutigen Geburtstag! https://t.co/JIwPQwKZpp,wir gratulieren unserem bundesverkehrsminister andreas scheuer herzlichst zu seinem heutigen geburtstag httpstcojiwpqwkzpp,bundesverkehrsminister heutig geburtstag
1,CSU,An alle unsere Wahlkämpfer und Wähler: Danke für eure Unterstützung! #btw21 https://t.co/3lL8q9PJzR,An alle unsere Wahlkämpfer und Wähler: Danke für eure Unterstützung! #btw21 https://t.co/3lL8q9PJzR,an alle unsere wahlkämpfer und wähler danke für eure unterstützung,wahlkämpfer wähler unterstützung
2,CSU,Damit Bayern in Deutschland stark bleibt! Heute mit beiden Stimmen CSU! #btw21 https://t.co/WKjCFCmjiT,Damit Bayern in Deutschland stark bleibt! Heute mit beiden Stimmen CSU! #btw21 https://t.co/WKjCFCmjiT,damit bayern in deutschland stark bleibt heute mit beiden stimmen csu httpstcowkjcfcmjit,bayern stimmen
3,CSU,"Mit spitzer Feder gegen den Linksrutsch: Alexander Dobrindt trifft @janfleischhauer. Schaut rein in das Gespräch ÜBER statt UNTER LINKEN - und lernt von Jan Fleischhauer, was durchscholzen bedeutet. Jetzt unterstützen und Linksrutsch verhindern: https://t.co/fZun4YDRny https://t.co/8QXxDWPVr6","Mit spitzer Feder gegen den Linksrutsch: Alexander Dobrindt trifft @janfleischhauer. Schaut rein in das Gespräch ÜBER statt UNTER LINKEN - und lernt von Jan Fleischhauer, was durchscholzen bedeutet. Jetzt unterstützen und Linksrutsch verhindern: https://t.co/fZun4YDRny https://t.co/8QXxDWPVr6",mit spitzer feder gegen den linksrutsch alexander dobrindt trifft janfleischhauer schaut rein in das gespräch über statt unter linken und lernt von jan fleischhauer was durchscholzen bedeutet jetzt unterstützen und linksrutsch verhindern,spitzer feder linksrutsch gespräch link durchscholzen linksrutsch
4,CSU,"Wir wollen, dass Armin Laschet der nächste Kanzler wird! Damit Deutschland stabil bleibt! #btw21 https://t.co/ZIP4PsASsQ","Wir wollen, dass Armin Laschet der nächste Kanzler wird! Damit Deutschland stabil bleibt! #btw21 https://t.co/ZIP4PsASsQ",wir wollen dass armin laschet der nächste kanzler wird damit deutschland stabil bleibt,armin nächst kanzler
...,...,...,...,...,...
317,CSU,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,Gedenkminute im Stadion für die Opfer der entsetzlichen Messerattacke in #Würzburg. Wir sind in Gedanken bei den Familien. https://t.co/vEginx1LxT,gedenkminute im stadion für die opfer der entsetzlichen messerattacke in würzburg wir sind in gedanken bei den familien,gedenkminute stadion opfer entsetzlich messerattacke gedanken familien
318,CSU,🔵 Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. ⬇️ https://t.co/yk6Nvr90xW,Jetzt LIVE: die Aufstellung der #CSU-Liste für die Bundestagswahl aus dem Max-Morlock-Stadtion in Nürnberg. https://t.co/yk6Nvr90xW,jetzt live die aufstellung der csuliste für die bundestagswahl aus dem maxmorlockstadtion in nürnberg,aufstellung csuliste bundestagswahl maxmorlockstadtion
319,CSU,".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so etwas in Zukunft zu verhindern! #Würzburg",".@Markus_Soeder: Ich danke der #Polizei für ihren schnellen Einsatz. Besonders beeindruckt hat mich das Engagement der Bürger! Wir stehen nicht nur den Betroffenen bei, sondern werden alles tun, um so etwas in Zukunft zu verhindern! #Würzburg",markussoeder ich danke der polizei für ihren schnellen einsatz besonders beeindruckt hat mich das engagement der bürger wir stehen nicht nur den betroffenen bei sondern

In [62]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','markusblume','markussoeder'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [64]:
unigram

,unigram,count
0,land,35
1,union,25
2,regierungserklärung,22
3,bayern,20
4,link,19
5,livestream,18
6,linksrutsch,17
7,stark,16
8,klimaschutz,15
9,herzlich,14


In [65]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [66]:
bigram

,bigram,count
0,herzlich glückwunsch,12
1,jahre spitz,9
2,spitz erfolg,9
3,weit jahre,8
4,bezirksverband weit,6
5,frei wähler,5
6,groß csustadiontour,5
7,stabilität linksrutsch,5
8,csuchef generalsekretär,4
9,erfolg politik,4


In [67]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [68]:
trigram

,trigram,count
0,jahre spitz erfolg,9
1,weit jahre spitz,8
2,bezirksverband weit jahre,6
3,union stark kraft,3
4,csuchef generalsekretär spitzenkandidaten,3
5,heiß phase wahlkampfs,3
6,spitz erfolg politik,3
7,herzlich glückwunsch bezirksverband,3
8,steuererhöhungen aufschwung gegenteil,2
9,aufschwung gegenteil absteigen,2


In [69]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<322x418 sparse matrix of type '<class 'numpy.int64'>'
	with 1597 stored elements in Compressed Sparse Row format>

In [72]:
lda_model = LatentDirichletAllocation(
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=0,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [73]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['groß', 'grün', 'menschen', 'bayerisch', 'geburtstag', 'stark', 'bayern', 'partei', 'land', 'union']


Top 10 words for topic #1:
['weit jahre', 'weit', 'spitz erfolg', 'spitz', 'jahre spitz', 'jahre', 'herzlich', 'herzlich glückwunsch', 'glückwunsch', 'erfolg']


Top 10 words for topic #2:
['millionen', 'gut', 'bundeswehr', 'schritt', 'bundestagswahl', 'livestream', 'familien', 'soldaten', 'freiheit', 'land']


Top 10 words for topic #3:
['pandemie', 'erhalt', 'wohlstand', 'lockdown', 'bayern', 'gillamoos', 'generalsekretär', 'livestream', 'regierungserklärung', 'klimaschutz']


Top 10 words for topic #4:
['csuprogramm', 'hoch', 'afd', 'regierung', 'generalsekretär', 'linksrutsch', 'pressekonferenz', 'land', 'uhr', 'link']




In [47]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builti

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  24.504876
3       51.163364 -159.968018       2        1  22.031210
0      -73.144554   21.523815       3        1  20.725369
4      -66.410133 -110.606438       4        1  18.189933
2       54.417324   12.443444       5        1  14.548613, topic_info=                    Term       Freq      Total Category  logprob  loglift
303  regierungserklärung  18.000000  18.000000  Default  30.0000  30.0000
210                 land  28.000000  28.000000  Default  29.0000  29.0000
170             herzlich  12.000000  12.000000  Default  28.0000  28.0000
95                erfolg  11.000000  11.000000  Default  27.0000  27.0000
47        bundestagswahl  11.000000  11.000000  Default  26.0000  26.0000
..                   ...        ...        ...      ...      ...      ...
375                union   1.788768  20.909020   Topic5  -4.8999  -0.5310
349               starke   1.724572   4.759057   Topic5  -4.9365   0.9126
221                letzt   1.724094   5.575901   Topic5  -4.9368   0.7539
22               aufgabe   1.719955   4.709486   Topic5  -4.9392   0.9204
56             csubtchef   1.719515   4.709422   Topic5  -4.9394   0.9202

[272 rows x 6 columns], token_table=      Topic      Freq                      Term
term                                           
3         4  0.837589                 absteigen
4         4  0.837589             absteigen neu
5         1  0.466685                       afd
5         2  0.155562                       afd
5         4  0.311123                       afd
...     ...       ...                       ...
410       5  0.847839                    zeiten
411       1  0.817830                      ziel
415       2  0.936608               zweitstimme
416       2  0.936608  zweitstimme bayernstimme
417       1  0.922215                ökologisch

[358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 3])